![Xilinx Logo](images/xilinx_logo.png "Xilinx Logo")

# 1. Introduction

This notebook demonstrates how to run the defect detection pipeline in different modes like file and live capture. Live images are captured from the MIPI device, then performs defect detection acceleration using Vitis Vision library. The defect results text will be embedded along with the accelerator output image and sends out to display.


# 2. Imports and Initialization
Import all python modules required for this notebook.


In [1]:
import sys
import glob
import subprocess
import re

In [2]:
import gi
gi.require_version('Gst', '1.0')
gi.require_version("GstApp", "1.0")
gi.require_version('GstVideo', '1.0')
gi.require_version('GIRepository', '2.0')
from gi.repository import GObject, Gst, GstVideo, GLib, GIRepository
Gst.init(None)

[]

In [3]:
mainloop = GLib.MainLoop()


## 3. Run the modetest command to configure the mixer IP for 4K

In [4]:
%%bash
modetest -M xlnx -D B0010000.v_mix -s 52@40:3840x2160@NV16  


setting mode 3840x2160-30.00Hz@NV16 on connectors 52, crtc 40


# 4. Construct the String Representation of GStreamer Pipeline
## The get_media_by_device function returns the matching media node for a given video capture source. The following sources are supported in this notebook:

* mipi : platform1 only

In [5]:
def get_media_dev_by_name(src):
    sources = {
        'mipi' : 'vcap_csi',
    }
    devices = glob.glob('/dev/media*')
    for dev in devices:
        proc = subprocess.run(['media-ctl', '-d', dev, '-p'], capture_output=True, encoding='utf8')
        for line in proc.stdout.splitlines():
            if sources[src] in line:
                return dev

## Change the DEMO variable to switch between filesrc/media soruce
## Change the filesinkpath to store the different stages output

In [6]:
# 0 for file input and file sinc
# 1 for media src bin
DEMO = 0
filesinkpath="/home/petalinux/"

## Get the source by calling get_media_dev_by_name()
## For file source copy the input video file to any rootfs path & update the location variable accordingly 

In [7]:
source = "mipi" # Change source usb

media_device = get_media_dev_by_name(source) 
if media_device is None:
    raise Exception('Unable to find video source ' + source + '. Make sure the device is plugged in, powered, and the correct platform is used.')

src = "gst-launch-1.0 "

if DEMO == 1:#media src bin
    src += "mediasrcbin media-device=" + media_device
else:# file src
    src += "filesrc location=/home/petalinux/input_video.y8 blocksize=1024000"    

print(src)

gst-launch-1.0 filesrc location=/home/petalinux/input_video.y8 blocksize=1024000


## Construct the real pipeline string.

### configuration directory for IVAS plugin

In [8]:
confdir="/opt/xilinx/share/ivas/defect-detect"

### Set the caps.
User can change the resolution and framerate here.

If videosrc cannot support format GRAY8, adjust the pipeline to fit with followning elements.

In [9]:
pip=src + ' ! video/x-raw, width=1280, height=800, format=GRAY8, framerate=60/1 '
print(pip)

gst-launch-1.0 filesrc location=/home/petalinux/input_video.y8 blocksize=1024000 ! video/x-raw, width=1280, height=800, format=GRAY8, framerate=60/1 


### Run the live capture pipeline.
ivas_xfilter kconfig="{confdir}/pre-process.json" is configure the threshold & max value, kernel name, xclbin path.
canny-accelarator.json to cinfigure min & max threshold value.
dfeect-calculation.json is to configure threshold value to decide whether fruit is defect or not, font size, font.

Detailed configuration can be found in the json file.

In [10]:
# pip += ' ! tee name=t_src t_src. ! queue ! ivas_xfilter kernels-config={confdir}/pre-process.json ! tee name=t_pre t_pre. ! queue ! ivas_xfilter kernels-config={confdir}/canny-accelarator.json ! ivas_xfilter kernels-config={confdir}/edge-tracer.json ! ivas_xfilter kernels-config={confdir}/defect-calculation.json  ! perf ! kmssink  bus-id=B0010000.v_mix  plane-id=34 render-rectangle="<2560,0,1280,800>"  t_src. !  queue ! perf ! kmssink bus-id=B0010000.v_mix  plane-id=35 render-rectangle="<0, 0,1280,800>" async=false  t_pre. ! queue ! perf ! kmssink bus-id=B0010000.v_mix  plane-id=36 render-rectangle="<1280,0,1280,800>" async=false '.format(confdir=confdir)
# pip += ' ! tee name=t_src t_src. ! queue ! ivas_xfilter kernels-config=pre-process.json ! tee name=t_pre t_pre. ! queue ! ivas_xfilter kernels-config=canny-accelarator.json ! ivas_xfilter kernels-config=edge-tracer.json ! ivas_xfilter kernels-config=defect-calculation.json  ! videorate !  "video/x-raw, framerate=4/1" !  perf ! kmssink  bus-id=B0010000.v_mix  plane-id=34 render-rectangle="<2560,0,1280,800>"  t_src. !  queue ! videorate !  "video/x-raw, framerate=4/1" ! perf ! kmssink bus-id=B0010000.v_mix  plane-id=35 render-rectangle="<0,0,1280,800>" async=false  t_pre. ! queue ! videorate !  "video/x-raw, framerate=4/1" ! perf ! kmssink bus-id=B0010000.v_mix  plane-id=36 render-rectangle="<1280,0,1280,800>" async=false' 
#pip = 'gst-launch-1.0 mediasrcbin media-device=/dev/media0 ! video/x-raw, width=1280, height=800, format=GRAY8, framerate=60/1  ! tee name=t_src t_src. ! queue ! ivas_xfilter kernels-config={confdir}/pre-process.json ! tee name=t_pre t_pre. ! queue ! ivas_xfilter kernels-config={confdir}/canny-accelarator.json ! ivas_xfilter kernels-config={confdir}/edge-tracer.json ! ivas_xfilter kernels-config={confdir}/defect-calculation.json  ! videorate !  video/x-raw, framerate=4/1 !  perf ! kmssink  bus-id=B0010000.v_mix  plane-id=34 render-rectangle="<2560,0,1280,800>"  t_src. !  queue ! videorate !  video/x-raw, framerate=4/1 ! perf ! kmssink bus-id=B0010000.v_mix  plane-id=35 render-rectangle="<0,0,1280,800>" async=false  t_pre. ! queue ! videorate !  video/x-raw, framerate=4/1 ! perf ! kmssink bus-id=B0010000.v_mix  plane-id=36 render-rectangle="<1280,0,1280,800>" async=false '.format(confdir=confdir)
filesink = (DEMO == 0)

pip += ' ! tee name=t_src t_src. ! queue ! ivas_xfilter kernels-config={confdir}/pre-process.json ! tee name=t_pre t_pre. ! queue ! ivas_xfilter kernels-config={confdir}/canny-accelarator.json ! ivas_xfilter kernels-config={confdir}/edge-tracer.json ! ivas_xfilter kernels-config={confdir}/defect-calculation.json  '.format(confdir=confdir)

if filesink:
    pip += "! filesink  location={path}/final.yuv".format(path=filesinkpath)
else:
    pip += '! videorate !  video/x-raw, framerate=4/1 !  perf ! kmssink  bus-id=B0010000.v_mix  plane-id=34 render-rectangle="<2560,0,1280,800>"'

pip += '  t_src. !  queue '

if filesink:
    pip += "! filesink location={path}/raw_src.yuv".format(path=filesinkpath)
else:
    pip += '! videorate !  video/x-raw, framerate=4/1 ! perf ! kmssink bus-id=B0010000.v_mix  plane-id=35 render-rectangle="<0,0,1280,800>"'

pip += ' async=false  t_pre. ! queue ! '

if filesink:
    pip += "filesink location={path}/pre-process.yuv  async=false ".format(path=filesinkpath)
else:
    pip += 'videorate !  video/x-raw, framerate=4/1 ! perf ! kmssink bus-id=B0010000.v_mix  plane-id=36 render-rectangle="<1280,0,1280,800>" async=false '.format(confdir=confdir)

print(pip)

gst-launch-1.0 filesrc location=/home/petalinux/input_video.y8 blocksize=1024000 ! video/x-raw, width=1280, height=800, format=GRAY8, framerate=60/1  ! tee name=t_src t_src. ! queue ! ivas_xfilter kernels-config=/opt/xilinx/share/ivas/defect-detect/pre-process.json ! tee name=t_pre t_pre. ! queue ! ivas_xfilter kernels-config=/opt/xilinx/share/ivas/defect-detect/canny-accelarator.json ! ivas_xfilter kernels-config=/opt/xilinx/share/ivas/defect-detect/edge-tracer.json ! ivas_xfilter kernels-config=/opt/xilinx/share/ivas/defect-detect/defect-calculation.json  ! filesink  location=/home/petalinux//final.yuv  t_src. !  queue ! filesink location=/home/petalinux//raw_src.yuv async=false  t_pre. ! queue ! filesink location=/home/petalinux//pre-process.yuv  async=false 


In [ ]:
output=subprocess.check_output(pip,shell=True)

print (output)

# 5. Summary
The Jupyter application shows how to:

1. Create a GStreamer pipeline which utilize the IVAS framework to call Vitis Vision Library to do defect detection on the live mango images, and embed the defect result on the final image from accelerator.
2. User can try to customize the source to video file or MIPI camera.

<center>Copyright© 2019 Xilinx</center>